# Day 9: Coding Rotary Positional Embeddings (RoPE)

## Quick Recap: What is RoPE? 
- RoPE is a method for injection positional information into tranformer models, not by adding position vectors (like absolute positional embeddings), but by rotating the query and key vectors within the attention mechanism. 
- This provides several advantages: 
    - Relative Position Awareness: Understands the distance between tokens.
    - Extrapolation: Handles sequences longer than seen during training. 
    - Efficiency: Doesn't require additional embeddings -just math inside attention 

## Code Walkthrough 
-  Let’s walk through how RoPE is implemented in the DeepSeek-Children-Stories-15M-model https://github.com/ideaweaver-ai/DeepSeek-Children-Stories-15M-model codebase. 

### 1. Implementation: RoPEPositionalEncoding 
- In the file src/model/deepseek.py, we will find the class RoPEPositionalEncoding. 
- This class:  
    - Precomputes rotation frequencies 
    - Provides an apply_rope method. 
    -Applies RoPE to input tensors, usually the query and key vectors. 

In [ ]:
# deepseek.py 
class RoPEPositionalEncoding(nn.Module):
    def __init__(self, dim, max_len=2048):
        super().__init__()
        inv_freq= 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        t = torch.arange(max_len, dtype=torch.float)
        freqs = torch.einsum("i,j->ij", t, inv_freq)
        emb = torch.cat((freqs.sin(), freqs.cos()), dim=-1)
        self.register_buffer("positional encoding", emb)

    def apply_rope(self, x, position_ids):
        rope = self.positional_encoding[position_ids]
        x1, x2 = x[..., ::2], x[..., 1::2]
        rope1, rope2 = rope[..., ::2], rope[..., 1::2]
        return torch.cat([x1 * rope2 + x2 * rope1, x2 * rope2 - x1 * rope1], dim=-1)

- The key idea is rotating even and odd dimensions of the query/key vectors based on sine and cosine frequencies. 

### 2: Usage: Integrating RoPE into Attention 
- The DeepSeek model utilizes a custom attention mechanism known as Multihead Latent Attention (MLA). 
- Here's how RoPE is integrated: 

In [ ]:
# deepseek.py 
q = self.q_proj(x)
k = self.k_proj(x)

q = self.rope.apply_rope(q, positions_ids)
k = self.rope.apply_rope(k, position_ids)

- What's happening? 
    - x is projected into query (q) and key (k) vectors. 
    - RoPE is applied to both using apply_rope, injecting position awarenness. 
    - Attention proceeds as usual --except now the queries and keys are aware of their relative positions. 

### 3: Where RoPe is used 
- Every Transformer Block: Each block in the DeepSeek model uses MLA and applies RoPE. 
- During Both Training and Inference: RoPE is always on, helping the model understand the token sequences no matter the mode.

## Why RoPE is Perfect for Story Generation 
- In story generation, especially for children's stories, context is everything. 
- RoPE enables the model to:    
    - Track who did what across paragraphs 
    - Maintain chronological consistency
    - Preserve narrative flow even in long outputs.

# Conclusion
- Rotary Positional Embeddings (RoPE) are not just a theoratical improvement; they offer practical performance and generalization benefits. 
-  If we are working on any transformer-based task with long sequences, story generation, document QA, or chat history modeling, we should absolutely consider using RoPE.